In [89]:
# %load HMM.py
from __future__ import print_function

from tabulate import tabulate
import numpy as np
import pdb


class HMM(object):

    def __init__(self, A, B, pi0=None, states=None, emissions=None):
        """
        :param A: Transition matrix of shape (n, n) (n = number of states)
        :param B: Emission matrix of shape (n, b) (b = number of outputs)
        :param pi0: Initial State Probability vector of size n, leave blank for uniform probabilities
        :param states: State names/labels as list
        :param emissions: Emission names/labels as list
        """
        self.A = A
        self.B = B
        self.n_states = A.shape[0]
        self.n_emissions = B.shape[1]
        self.states = states
        self.emissions = emissions
        self.pi0 = pi0

        if pi0 is None:
            self.pi0 = np.full(self.n_states, 1.0 / self.n_states)

        if states is None:
            self.states = [chr(ord('A') + i) for i in range(self.n_states)]

        if emissions is None:
            self.emissions = [str(i) for i in range(self.n_emissions)]
        
    def print_matrix(self, M, headers=None):
        """
        Print matrix in tabular form

        :param M: Matrix to print
        :param headers: Optional headers for columns, default is state names
        :return: tabulated encoding of input matrix
        """
        headers = headers or self.states

        if M.ndim > 1:
            headers = [' '] + headers
            data = [['t={}'.format(i + 1)] + [j for j in row] for i, row in enumerate(M)]
        else:
            data = [[j for j in M]]
        print(tabulate(data, headers, tablefmt="grid", numalign="right"))
        return None
        
    def forward_algorithm(self, seq):
        
        
        """
        Apply forward algorithm to calculate probabilities of seq

        :param seq: Observed sequence to calculate probabilities upon
        :return: Alpha matrix with 1 row per time step
        """
        Alphas = []
        Zs = []
        
        T = len(seq)

        # Initialize forward probabilities matrix Alpha
        Alpha = np.zeros((T, self.n_states))
        
        a = self.B.transpose()[seq[0]] * self.pi0
        Z = sum(a)
        a = a / Z
        
        Alpha[0] = a
        #Zs.append(np.log(Z))
        
        # Your implementation here
        for t in range(1, T):
            a = self.B.transpose()[seq[t]] * np.matmul(np.transpose(self.A),a)
            Z = sum(a)
            a = a / Z

            Alpha[t] = a
            #Zs.append(np.log(Z))
        
        return Alpha#, sum(Zs)    

    def backward_algorithm(self, seq):
        """
        Apply backward algorithm to calculate probabilities of seq

        :param seq: Observed sequence to calculate probabilities upon
        :return: Beta matrix with 1 row per timestep
        """

        T = len(seq)

        ## Initialize backward probabilities matrix Beta
        Beta = np.zeros((T, self.n_states))
        
        b = np.ones(self.n_states)
        Beta[T-1] = b
        # Your implementation here
        
        for t in range(T-1, 0, -1):
            b = np.matmul(self.A, self.B.transpose()[seq[t]] * b)
            Beta[t-1] = b
            
        return Beta

    def forward_backward(self, seq):
        """
        Applies forward-backward algorithm to seq

        :param seq: Observed sequence to calculate probabilities upon
        :return: Gamma matrix containing state probabilities for each timestamp
        :raises: ValueError on bad sequence
        """

        # Convert sequence to integers
        if all(isinstance(i, str) for i in seq):
            seq = [self.emissions.index(i) for i in seq]

        # Infer time steps
        T = len(seq)
        
        # Calculate forward probabilities matrix Alpha
        Alpha = self.forward_algorithm(seq)
        # Initialize backward probabilities matrix Beta
        Beta = self.backward_algorithm(seq)

        # Initialize Gamma matrix
        Gamma = np.zeros((T, self.n_states))        
        
        # Your implementation here
        Gamma = Alpha * Beta
        for i in Gamma:
            i /= sum(i)
        
        print('Alpha:')
        print(Alpha)
        print('Beta:')
        print(Beta)
        print('Gamma:')
        print(Gamma)
        
        return Gamma

In [90]:
# %load HMM_example.py
from __future__ import print_function
import numpy as np

#from HMM import HMM

"""
Example from ICA 4
"""

A = np.array([[0.40, 0.60],
              [0.80, 0.20]])

B = np.array([[0.40, 0.60],
              [0.70, 0.30]])

pi0 = np.array([0.90, 0.10])


seq = ['WB', 'PS', 'WB']

model = HMM(A, B, pi0,
            states=['BS', 'CS'],
            emissions=['PS', 'WB'])

res = model.forward_backward(seq)
model.print_matrix(res)


Alpha:
[[0.94736842 0.05263158]
 [0.29357798 0.70642202]
 [0.81134133 0.18865867]]
Beta:
[[0.294 0.21 ]
 [0.42  0.54 ]
 [1.    1.   ]]
Gamma:
[[0.96183206 0.03816794]
 [0.24427481 0.75572519]
 [0.81134133 0.18865867]]
+-----+----------+-----------+
|     |       BS |        CS |
+=====+==========+===========+
| t=1 | 0.961832 | 0.0381679 |
+-----+----------+-----------+
| t=2 | 0.244275 |  0.755725 |
+-----+----------+-----------+
| t=3 | 0.811341 |  0.188659 |
+-----+----------+-----------+


In [91]:
# %load HMM_example.py
from __future__ import print_function
import numpy as np

#from HMM import HMM

"""
Example from ICA 4
"""

A = np.array([[0.15, 0.25, 0.25, 0.35],
              [0.6, 0.2, 0.1, 0.1],
              [0.25, 0.2, 0.3, 0.25],
              [0.1, 0.4, 0.4, 0.1]])

B = np.array([[0.6, 0.1, 0.1, 0.1, 0.1],
              [0.1, 0.6, 0.1, 0.1, 0.1],
              [0.1, 0.2, 0.2, 0.2, 0.3],
              [0, 0, 0, 0.5, 0.5]])

pi0 = np.array([.25,.25,.25,.25])


seq = ['e4', 'e3', 'e2', 'e2', 'e0', 'e1']

model = HMM(A, B, pi0,
            states=['A', 'B', 'C', 'D'],
            emissions=['e0', 'e1','e2', 'e3', 'e4'])

res = model.forward_backward(seq)
model.print_matrix(res)

Alpha:
[[0.1        0.1        0.3        0.5       ]
 [0.09975062 0.1521197  0.32418953 0.42394015]
 [0.20264026 0.25566557 0.54169417 0.        ]
 [0.31705638 0.20870848 0.47423514 0.        ]
 [0.79229672 0.09783421 0.10986907 0.        ]
 [0.09650644 0.67677038 0.22672317 0.        ]]
Beta:
[[1.50332603e-04 7.54065375e-05 1.26552553e-04 9.33956062e-05]
 [4.24356875e-04 4.63610000e-04 4.98911875e-04 6.13893750e-04]
 [4.64837500e-03 4.28550000e-03 4.94987500e-03 7.02475000e-03]
 [2.94750000e-02 8.34500000e-02 4.24000000e-02 2.91000000e-02]
 [2.15000000e-01 2.00000000e-01 2.05000000e-01 3.30000000e-01]
 [1.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00]]
Gamma:
[[0.14018662 0.07031733 0.35403447 0.43546157]
 [0.07914339 0.13185786 0.30240613 0.48659263]
 [0.19961087 0.23218332 0.56820581 0.        ]
 [0.19938832 0.37160012 0.42901156 0.        ]
 [0.80186767 0.09210795 0.10602437 0.        ]
 [0.09650644 0.67677038 0.22672317 0.        ]]
+-----+-----------+-----------+----